# Stanza pipeline

If `gatenlp` has been installed with the stanza extra (`pip install gatenlp[stanza]` or `pip install gatenlp[all]`) you can run a Stanford Stanza pipeline on a document and get the result as `gatenlp` annotations. 



In [1]:
from gatenlp import Document
from gatenlp.lib_stanza import AnnStanza
import stanza



In [3]:
# In order to use the English pipeline with stanza, the model has to get downloaded first
stanza.download('en')

INFO:stanza:Downloading default packages for language: en (English)...
INFO:stanza:File exists: /home/johann/stanza_resources/en/default.zip.
INFO:stanza:Finished downloading models and saved to /home/johann/stanza_resources.


In [2]:
doc = Document.load("https://gatenlp.github.io/python-gatenlp/testdocument1.txt")

Document(This is a test document.

It contains just a few sentences. 
Here is a sentence that mentions a few named entities like 
the persons Barack Obama or Ursula von der Leyen, locations
like New York City, Vienna or Beijing or companies like 
Google, UniCredit or Huawei. 

Here we include a URL https://gatenlp.github.io/python-gatenlp/ 
and a fake email address john.doe@hiscoolserver.com as well 
as #some #cool #hastags and a bunch of emojis like 😽 (a kissing cat),
👩‍🏫 (a woman teacher), 🧬 (DNA), 
🧗 (a person climbing), 
💩 (a pile of poo). 

Here we test a few different scripts, e.g. Hangul 한글 or 
simplified Hanzi 汉字 or Farsi فارسی which goes from right to left. 


,features=Features({}),anns=[])


Printing the document shows the document text and indicates that there are no document features and no 
annotations which is to be expected since we just loaded from a plain text file. 

In a Jupyter notebook, a `gatenlp` document can also be visualized graphically by either just using the document 
as the last value of a cell or by using the IPython "display" function:

In [3]:
from IPython.display import display
display(doc)

Document(This is a test document.

It contains just a few sentences. 
Here is a sentence that mentions a few named entities like 
the persons Barack Obama or Ursula von der Leyen, locations
like New York City, Vienna or Beijing or companies like 
Google, UniCredit or Huawei. 

Here we include a URL https://gatenlp.github.io/python-gatenlp/ 
and a fake email address john.doe@hiscoolserver.com as well 
as #some #cool #hastags and a bunch of emojis like 😽 (a kissing cat),
👩‍🏫 (a woman teacher), 🧬 (DNA), 
🧗 (a person climbing), 
💩 (a pile of poo). 

Here we test a few different scripts, e.g. Hangul 한글 or 
simplified Hanzi 汉字 or Farsi فارسی which goes from right to left. 


,features=Features({}),anns={})

This shows the document in a layout that has three areas: the document text in the upper left,
the list of annotation set and type names in the upper right and document or annotation features
at the bottom. In the example above only the text is shown because there are no document features or 
annotations. 

## Document features

Lets add some document features:

In [4]:
doc.features["loaded-from"] = "https://gatenlp.github.io/python-gatenlp/testdocument1.txt"
doc.features["purpose"] = "test document for gatenlp"
doc.features["someotherfeature"] = 22
doc.features["andanother"] = {"what": "a dict", "alist": [1,2,3,4,5]}

Document features map feature names to feature values and behave a lot like a Python dictionary. Feature names
should always be strings, feature values can be anything, but a document can only be stored or exchanged with Java GATE if feature values are restricted to whatever can be serialized with JSON: dictionaries, lists, numbers, strings and booleans. 

Now that we have create document features the document is shown like this:

In [5]:
doc

Document(This is a test document.

It contains just a few sentences. 
Here is a sentence that mentions a few named entities like 
the persons Barack Obama or Ursula von der Leyen, locations
like New York City, Vienna or Beijing or companies like 
Google, UniCredit or Huawei. 

Here we include a URL https://gatenlp.github.io/python-gatenlp/ 
and a fake email address john.doe@hiscoolserver.com as well 
as #some #cool #hastags and a bunch of emojis like 😽 (a kissing cat),
👩‍🏫 (a woman teacher), 🧬 (DNA), 
🧗 (a person climbing), 
💩 (a pile of poo). 

Here we test a few different scripts, e.g. Hangul 한글 or 
simplified Hanzi 汉字 or Farsi فارسی which goes from right to left. 


,features=Features({'loaded-from': 'https://gatenlp.github.io/python-gatenlp/testdocument1.txt', 'purpose': 'test document for gatenlp', 'someotherfeature': 22, 'andanother': {'what': 'a dict', 'alist': [1, 2, 3, 4, 5]}}),anns={})

In [6]:
# to retrieve a feature value we can do:
doc.features["purpose"]

'test document for gatenlp'

In [7]:
# If a feature does not exist, None is returned or a default value if specified:
print(doc.features.get("doesntexist"))
print(doc.features.get("doesntexist", "MV!"))


None
MV!


## Annotations

Lets add some annotations too. Annotations are items of information for some range of characters within the document. They can be used to represent information about things like tokens, entities, sentences, paragraphs, or 
anything that corresponds to some contiguous range of offsets in the document.

Annotations consist of the following parts:
* The "start" and "end" offset to identify the text the annotation refers to
* A "type" which is an arbitrary name that identifies what kind of thing the annotation describes, e.g. "Token"
* Features: these work in the same way as for the whole document: an arbitrary set of feature name / feature value
  pairs which provide more information, e.g. for a Token the features could include the lemma, the part of speech,
  the stem, the number, etc. 

Annotations can be organized in "annotation sets". Each annotation set has a name and a set of annotations. There can be as many sets as needed. 

Annotation can overlap arbitrarily and there can be as many as needed. 

Let us manually add a few annotations to the document:

In [8]:
# create and get an annotation set with the name "Set1"
annset = doc.annset("Set1")

Add an annotation to the set which refers to the first word in the document "This". The range of characters
for this word starts at offset 0 and the length of the annotation is 4, so the "start" offset is 0 and the "end" offset is 0+4=4. Note that the end offset always points to the offset *after* the last character of the range.

In [9]:
annset.add(0,4,"Word",{"what": "our first annotation"})

Annotation(0,4,Word,features=Features({'what': 'our first annotation'}),id=0)

In [10]:
# Add more
annset.add(5,7,"Word",{"what": "our second annotation"})
annset.add(0,24,"Sentence",{"what": "our first sentence annotation"})

Annotation(0,24,Sentence,features=Features({'what': 'our first sentence annotation'}),id=2)

If we visualize the document now, the newly created set "Set" is shown in the right part of
the display. It shows the different annotation types that exist in the set, and how many annotations
for each type are in the set. If you click the check box, the annotation ranges are shown in the 
text with the colour associated with the annotation type. You can then click on a range / annotation in the
text and the features of the annotation are shown in the lower part. 
To show the features for a different annotation click on the coloured range for the annotation in the text.
To show the document features, click on "Document".

If you have selected more than one type, a range can have more than one overlapping annotations. 
This is shown by mixing the colours. If you click at such a location, a dialog appears which lets you
select for which of the overlapping annotations you want to display the features. 

In [11]:
doc

Document(This is a test document.

It contains just a few sentences. 
Here is a sentence that mentions a few named entities like 
the persons Barack Obama or Ursula von der Leyen, locations
like New York City, Vienna or Beijing or companies like 
Google, UniCredit or Huawei. 

Here we include a URL https://gatenlp.github.io/python-gatenlp/ 
and a fake email address john.doe@hiscoolserver.com as well 
as #some #cool #hastags and a bunch of emojis like 😽 (a kissing cat),
👩‍🏫 (a woman teacher), 🧬 (DNA), 
🧗 (a person climbing), 
💩 (a pile of poo). 

Here we test a few different scripts, e.g. Hangul 한글 or 
simplified Hanzi 汉字 or Farsi فارسی which goes from right to left. 


,features=Features({'loaded-from': 'https://gatenlp.github.io/python-gatenlp/testdocument1.txt', 'purpose': 'test document for gatenlp', 'someotherfeature': 22, 'andanother': {'what': 'a dict', 'alist': [1, 2, 3, 4, 5]}}),anns={'Set1': AnnotationSet([Annotation(0,4,Word,features=Features({'what': 'our first annotation'}),id=0), Annotation(0,24,Sentence,features=Features({'what': 'our first sentence annotation'}),id=2), Annotation(5,7,Word,features=Features({'what': 'our second annotation'}),id=1)])})

# Loading a larger document

Lets load a larger document, and from an HTML file: the Wikipedia page for "Natural Language processing":



In [19]:
doc2 = Document.load("https://en.m.wikipedia.org/wiki/Natural_language_processing", fmt="html", parser="html.parser")
doc2

Document(
Natural language processing - Wikipedia
Open main menu
Home
Random
Nearby
Log in
Settings
Donate
About Wikipedia
Disclaimers
Search
Natural language processing
Language
Watch
Edit
Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.
 
An automated online assistant providing customer service on a web page, an example of an application where natural language processing is a major component.[1]
Challenges in natural language processing frequently involve speech recognition, natural language understanding, and natural-language generation.
Contents
1 History
1.1 Symbolic NLP (1950s - early 1990s)
1.2 Statistical NLP (1990s - 2010s)
1.3 Neural NLP (present)
2 Methods: Rules, statistics, neural networks
2.1 Statistical methods
2.2 Neural networks
3 Common NLP Tasks
3.1 Text and speech processing
3.2 Morphological analysis
3.3 Syntactic analysis
3.4 Lexical semantics (of individual words in context)
3.5 Relational semantics (semantics of individual sentences)
3.6 Discourse (semantics beyond individual sentences)
3.7 Higher-level NLP applications
4 Cognition and NLP
5 See also
6 References
7 Further reading
HistoryEdit
Further information: History of natural language processing
Natural language processing has its roots in the 1950s. Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, a task that involves the automated interpretation and generation of natural language, but at the time not articulated as a problem separate from artificial intelligence.
Symbolic NLP (1950s - early 1990s)Edit
The premise of symbolic NLP is well-summarized by John Searle's Chinese room experiment: Given a collection of rules (e.g., a Chinese phrasebook, with questions and matching answers), the computer emulates natural language understanding (or other NLP tasks) by applying those rules to the data it is confronted with.
1950s: The Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sentences into English. The authors claimed that within three or five years, machine translation would be a solved problem.[2]  However, real progress was much slower, and after the ALPAC report in 1966, which found that ten-year-long research had failed to fulfill the expectations, funding for machine translation was dramatically reduced.  Little further research in machine translation was conducted until the late 1980s when the first statistical machine translation systems were developed.
1960s: Some notably successful natural language processing systems developed in the 1960s were SHRDLU, a natural language system working in restricted "blocks worlds" with restricted vocabularies, and ELIZA, a simulation of a Rogerian psychotherapist, written by Joseph Weizenbaum between 1964 and 1966.  Using almost no information about human thought or emotion, ELIZA sometimes provided a startlingly human-like interaction. When the "patient" exceeded the very small knowledge base, ELIZA might provide a generic response, for example, responding to "My head hurts" with "Why do you say your head hurts?".
1970s: During the 1970s, many programmers began to write "conceptual ontologies", which structured real-world information into computer-understandable data.  Examples are MARGIE (Schank, 1975), SAM (Cullingford, 1978), PAM (Wilensky, 1978), TaleSpin (Meehan, 1976), QUALM (Lehnert, 1977), Politics (Carbonell, 1979), and Plot Units (Lehnert 1981).  During this time, the first many chatterbots were written (e.g., PARRY).
1980s: The 1980s and early 1990s mark the hey-day of symbolic methods in NLP. Focus areas of the time included research on rule-based parsing (e.g., the development of HPSG as a co

The markup present in the original HTML file is converted into annotations in the annotation set with the name "Original markups". For example all the HTML links are present as annotations of type "a" (there are 449 of those), the level 3 headings are present as annotaotions of type "h3" and so on. 



In [12]:

from gatenlp import lib_stanza
import stanza
# stanza.download("en")
nlp = stanza.Pipeline("en")


INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [13]:
doc2 = Document.load("https://en.m.wikipedia.org/wiki/Mike_Capel", fmt="html", parser="html.parser")



In [14]:
doc2 = lib_stanza.apply_stanza(nlp, doc2)

In [16]:
set1 = doc2.annset()
print("All: ",len(set1))
set_loc = set1.with_type("LOC")
print("LOC: ", len(set_loc))


All:  6820
LOC:  2


In [17]:
doc2

Document(
Mike Capel - Wikipedia
Open main menu
Home
Random
Nearby
Log in
Settings
Donate
About Wikipedia
Disclaimers
Search
Mike Capel
Language
Watch
Edit
Michael Lee Capel (born October 13, 1961) is an American professional baseball pitcher who played in Major League Baseball (MLB) for the Chicago Cubs, Milwaukee Brewers, and Houston Astros. In 49 career games, Capel pitched ​62 1⁄3 innings, struck out 43 batters, and had a career win–loss record of 3–4 with a 4.62 earned run average (ERA). While he played in MLB, Capel stood at 6 feet 1 inch (185 cm) and weighed 175 pounds (79 kg). A starting pitcher in college and parts of his minor league baseball career, he converted to relief pitching while in Chicago's minor league system.
Mike Capel
Pitcher
Born:  (1961-10-13) October 13, 1961 (age 59)
Marshall, Texas
Batted: Right
Threw: Right
MLB debut
May 7, 1988, for the Chicago Cubs
Last MLB appearance
August 14, 1991, for the Houston Astros
MLB statistics
Win–loss record3–4
Earned run average4.62
Strikeouts43
Teams
Chicago Cubs (1988)
Milwaukee Brewers (1990)
Houston Astros (1991)
The Philadelphia Phillies chose Capel in the 24th round of the 1980 MLB draft, but Capel did not sign with the team: instead, he opted to attend the University of Texas. Capel played on the 1982 USA College All-Star Team, which competed in the Amateur World Series in Seoul and placed third. The next year, Capel and the Texas Longhorns won the College World Series. After he was drafted by the Cubs, Capel left Texas and signed to play professional baseball; he played in six seasons of minor league baseball before he made his MLB debut in 1988. Capel spent the entire 1989 season in Triple-A, one level below the majors, but the Cubs released him at the end of the year. He agreed to terms with the Brewers and played in MLB after an injury opened a spot on Milwaukee's roster, but was again released at the end of the season. A free agent, the Astros signed Capel, and over the course of the season he pitched in 25 games for the team. He spent the final part of his career in the Astros farm system, and after he made the 1992 Triple-A All-Star team, Capel played his last season in 1993. After retirement, Capel worked as the general manager of a car dealership in Houston, Texas.
Contents
1 Early life
2 Collegiate career
3 Professional career
3.1 Chicago Cubs
3.2 Milwaukee Brewers
3.3 Houston Astros
4 Personal life
5 References
6 External links
Early life
Capel was born on October 13, 1961, in Marshall, Texas, and attended Spring High School in Harris County.[1] As a child, he enjoyed watching the Astros play on weekends.[2] During his senior year in 1979, the Spring Lions won the AAAA conference state championship, and Capel was named to the All-State team, composed of the best high school players in the state.[3][4] He set several pitching records at Spring High School.[5] Future teammate Calvin Schiraldi called Capel a "hard thrower when he came out of high school" and "the top guy out of the state in 1980";[6] Roger Clemens said he was "probably the best pitcher in the state at that time".[5] The Philadelphia Phillies drafted Capel with the 605th overall pick in the 1980 MLB Draft, but instead of signing with the Phillies, he chose to attend the University of Texas.[7][8]
Collegiate career
Capel enrolled at the University of Texas in 1981, but did not play baseball, as stretched ligaments in his elbow forced him to miss his freshman season. Capel pitched sidearm for the remainder of his collegiate career to compensate for a loss in the velocity of his fastball.[6] In 1982, he was named to the All-Southwest Conference team and pitched to a 9–0 win–loss record with a 3.68 ERA, as the Longhorns finished 59–6.[9][10] His 1982 winning percentage of 1.000 ties him with 10 others for the best single-season winning percentage in Longhorns history (with a minimum of nine decisions).[11] At the 1982 College World Series, Texas defeated the Oklahoma State Cowboys and the